# OrgaQuant - Batch Image Analysis
Quantify the organoid distribution from for all images in a folder and subfolders. This script uses a pre-trained deep learning model to locate the organoids and then quantify diameters (major and minor) and total organoid area. All output values are in units of 'pixels' these need to be scaled to the appropriate scale and units.

## Set all variables
* PATH_TO_CKPT:  Path to frozen detection graph you want to use. This is a pre-trained deep learning model (default is fine).
* PATH_TO_LABELS: List of the strings that is used to add correct label for each box (default is fine).
* FOLDER_PATH: Absolute path to the folder that contains the images and subfolder to quanitfy (this must be changed).
* look_pix: The size of the window, a good number is less than 600 pixels but bigger than an organoid (default is fine).
* slide_pix: To achieve redundancy and capture edge organoids, this is a sliding window (default is fine).

__Note: look_pix and slide_pix determine the total number of windows for each frame and hence the total time required to analyze a full image. Good numbers for look_pix is between 300-900 pixels and for slide_pix from 0.5 to 1 * look_pix.__

In [1]:
PATH_TO_CKPT = "C:/Users/Timothy/Desktop/OD/object_detection/organoid_inference_graph/frozen_inference_graph.pb"
PATH_TO_LABELS = "C:/Users/Timothy/Desktop/OD/object_detection/data/organoid_label_map.pbtxt"
FOLDER_PATH = "C:/Users/Timothy/Desktop/test_pro/"

## Import libraries

In [2]:
import sys
import os
import timeit
sys.path.append("..")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from utils import visualization_utils as vis_util
from organoid_utils import detect_all_organoids, get_metrics, detect_organoids


C:\Users\Timothy\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Get bounding boxes then save detection image and results
This takes several minutes to run. It returns organoids with a 95% confidence. Bounding boxes are drawn around the detected organoids. A png version of the image is also automatically saved in the same folder as the original image. The image is not needed for further anlysis but it is good to have it on record to confirm accurate detection.

In [3]:
for root, directories, filenames in os.walk(FOLDER_PATH):
    for i, filename in enumerate(filenames):
        
        try:
        
            print("Currently on {} of {} total files in {}".format(i+1, len(filenames), root))
            start_time = timeit.default_timer()

            IMAGE_PATH = os.path.join(root,filename)

            # Open image and load into numpy array
            full_image = Image.open(IMAGE_PATH)
            (full_im_width, full_im_height) = full_image.size
            full_image_np = np.array(full_image.getdata()).reshape((full_im_height, full_im_width, 3)).astype(np.uint8)


            [boxes, scores, classes, num, category_index] = detect_organoids(full_image_np,PATH_TO_CKPT, PATH_TO_LABELS, 1, thresh=0.95)


            vis_util.visualize_boxes_and_labels_on_image_array(
                full_image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=False,
                max_boxes_to_draw=None,
                min_score_thresh=0.00,
                line_thickness=3)

            plt.imsave(IMAGE_PATH + '_detected.png', full_image_np)

            # Get metrics
            [x1, y1, x2, y2, org_area, d_max, d_min] = get_metrics(boxes[0])

            elapsed_time = timeit.default_timer() - start_time

            # Save metrics
            results = pd.DataFrame(data={'xmin': x1,
                                         'ymin': y1,
                                         'xmax': x2,
                                         'ymax': y2,
                                         'area': org_area, 
                                         'max dia': d_max, 
                                         'min dia': d_min,
                                         'exec time': elapsed_time})

            results.to_csv(IMAGE_PATH + '.csv', index=False, columns=['xmin', 
                                                                      'ymin', 
                                                                      'xmax', 
                                                                      'ymax', 
                                                                      'area', 
                                                                      'max dia', 
                                                                      'min dia',
                                                                      'exec time'])
            print('Took {} seconds to analyze'.format(elapsed_time))
                              
        except:
            pass

Currently on 1 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 37.75039604938272 seconds to analyze
Currently on 2 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 31.725592888888897 seconds to analyze
Currently on 3 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 32.66607170370369 seconds to analyze
Currently on 4 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 31.61900958024691 seconds to analyze
Currently on 5 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 33.41613906172839 seconds to analyze
Currently on 6 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 34.81761698765433 seconds to analyze
Currently on 7 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 37.34552533333334 seconds to analyze
Currently on 8 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 34.05347160493827 seconds to analyze
Currently on 9 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 32.8092527

Took 35.25564760493853 seconds to analyze
Currently on 73 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 35.60953323456806 seconds to analyze
Currently on 74 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 32.98769422222222 seconds to analyze
Currently on 75 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 33.04252602469114 seconds to analyze
Currently on 76 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 37.38728493827148 seconds to analyze
Currently on 77 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 37.1473402469137 seconds to analyze
Currently on 78 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 35.73956424691369 seconds to analyze
Currently on 79 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 36.69490646913573 seconds to analyze
Currently on 80 of 112 total files in C:/Users/Timothy/Desktop/test_pro/
Took 34.71308879012349 seconds to analyze
Currently on 81 of 112 total files in C

In [4]:
boxes

array([[[1.4850816e+02, 4.7471952e+00, 2.3443105e+02, 9.1639076e+01],
        [2.7654724e+02, 1.6689479e+02, 3.6903574e+02, 2.6240280e+02],
        [2.0631566e+02, 1.8767564e+02, 2.4948575e+02, 2.2925281e+02],
        [1.2399149e+02, 5.4934059e+01, 2.0865485e+02, 1.3683754e+02],
        [6.8052017e+01, 2.0589418e+02, 1.4194238e+02, 2.7908078e+02],
        [3.0457825e+02, 5.4677650e+01, 3.4057758e+02, 9.3667915e+01],
        [1.8813339e+02, 2.5702527e+02, 2.3128511e+02, 3.0260577e+02],
        [2.9097562e+02, 2.9897229e+02, 3.4292136e+02, 3.4780334e+02],
        [3.9057827e-01, 1.7941022e+02, 5.7438549e+01, 2.4164114e+02],
        [1.1950304e+02, 3.7518460e+02, 1.5086349e+02, 4.0673044e+02],
        [2.3081384e+02, 2.6222662e+02, 2.6878528e+02, 3.0259946e+02],
        [2.6172546e+02, 1.4246106e+02, 3.2788464e+02, 2.0433801e+02],
        [5.0815830e+01, 1.4717455e+02, 7.6601364e+01, 1.7170625e+02],
        [3.7139087e+02, 1.3880042e+02, 4.2035327e+02, 1.8883202e+02],
        [1.4051622e+